In [1]:
"|IMPORT PACKAGES|"
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter
from   bokeh.models.tools import HoverTool

In [2]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'MONTHLY'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B, IU:IW", skiprows=2, nrows=396, engine='openpyxl') # Be patient...

data = data.dropna()

data = data.rename(columns={"Deuda Externa":"DEUDA_E"})
data = data.rename(columns={"Deuda Interna":"DEUDA_I"})
data = data.rename(columns={"TOTAL DEUDA":"DEUDA_T"})

In [3]:
"|CHECK DATA|"
data.tail()

,MONTH,DEUDA_E,DEUDA_I,DEUDA_T
383,2019-12-31,155.871719,164.757690,320.629409
386,2020-03-31,153.306434,167.652453,320.958887
389,2020-06-30,153.340000,167.700000,321.040000
392,2020-09-30,151.300000,170.800000,322.100000
395,2020-12-31,158.100000,171.700000,329.800000


In [5]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | DEUDA DE LA ADMINISTRACIÓN CENTRAL",
           x_axis_label = "Año",
           y_axis_label = "Miles de millones de US$", 
           plot_height  = 400,
           plot_width   = 700)

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1 = p.line("MONTH", "DEUDA_T", color="blue", line_alpha=0.50, width=4, legend_label="Deuda total"    , muted_alpha=0.20, source=cds)
p2 = p.line("MONTH", "DEUDA_E", color="red" , line_alpha=0.50, width=2, legend_label="Deuda externa"  , muted_alpha=0.20, source=cds)
p3 = p.line("MONTH", "DEUDA_I", color="grey", line_alpha=0.50, width=2, legend_label="Deuda domestica", muted_alpha=0.20, source=cds)

p.add_tools(HoverTool(renderers=[p1], tooltips = [("Deuda total"    , "@DEUDA_T")], mode="vline"))
p.add_tools(HoverTool(renderers=[p2], tooltips = [("Deuda externa"  , "@DEUDA_E")], mode="vline"))
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Deuda doméstica", "@DEUDA_I")], mode="vline"))


#LEGEND
p.legend.location     = "top_left"
p.legend.orientation  = "horizontal"
p.legend.click_policy = "mute"
show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="deuda_publica.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="deuda_publica.html", title="Deuda pública")
save(p)

In [6]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "deuda_publica"))
with open("deuda_publica.json", "w") as fp:
    json.dump(j, fp)